In [ ]:
import pandas as pd
import plotly.express as px

In [ ]:
metadata_path = '../../config/metadata_ms.tsv'
wkdir = '../..'
plate_info = False
cohort_cols = 'location,taxon'

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("max_colwidth", None)

# Run statistics

In [ ]:
cohort_col = cohort_cols.split(",")[0]

# load panel metadata
if metadata_path.endswith('.xlsx'):
    metadata = pd.read_excel(metadata_path, engine='openpyxl')
elif metadata_path.endswith('.tsv'):
    metadata = pd.read_csv(metadata_path, sep="\t")
elif metadata_path.endswith('.csv'):
    metadata = pd.read_csv(metadata_path, sep=",")
else:
    raise ValueError("Metadata file must be .xlsx or .csv")

import json
with open(f"{wkdir}/config/metadata_colours.json", 'r') as f:
    color_mapping = json.load(f)

demultiplex_data = pd.read_csv(
    f"{wkdir}/results/bcl_output/Reports/Demultiplex_Stats.csv"
).rename(columns={'# Reads':'n_reads', 'SampleID':'sample_id'})

What percentage of total reads are undetermined?

In [ ]:
n_undetermined = demultiplex_data.query("sample_id == 'Undetermined'")['n_reads'].to_list()[0]
pc_undetermined = demultiplex_data.query("sample_id == 'Undetermined'")['% Reads'].to_list()[0]

print(f"There are {n_undetermined:,} undetermined reads, {pc_undetermined*100}% of all reads")
print(f"There are {demultiplex_data['n_reads'].sum():,} total reads")

#### Total reads per sample

In [ ]:
demultiplex_data = demultiplex_data.query("sample_id != 'Undetermined'")
demultiplex_data = metadata.merge(demultiplex_data, how='left')

fig = px.bar(demultiplex_data, 
       x='sample_id', 
       y='n_reads', 
       color=cohort_col, 
       color_discrete_map=color_mapping[cohort_col],
       title='Total reads assigned to each sample',
       template='simple_white'
      )
fig.show()

#### Percentage of perfect and mismatched index reads

In [ ]:
fig = px.bar(demultiplex_data, 
       x='sample_id', 
       y='% Perfect Index Reads', 
       color=cohort_col, 
       color_discrete_map=color_mapping[cohort_col],
       title='The % of perfect index reads',
       template='simple_white'
      )
fig.show()

fig = px.bar(demultiplex_data.query("n_reads > 100"), 
       x='sample_id', 
       y='% One Mismatch Index Reads', 
       color=cohort_col, 
       color_discrete_map=color_mapping[cohort_col],
       title='The % of Index reads with one mismatch',
       template='simple_white'
      )
fig.show()

#### The number of reads for  each i7 index

In [ ]:
demultiplex_data.loc[:, 'i7'] = demultiplex_data['Index'].str.slice(0,8)
fig = px.box(demultiplex_data, 
             x='i7', 
             y='n_reads', 
             template='simple_white',
             color=cohort_col,
             color_discrete_map=color_mapping[cohort_col],
             title='Boxplot of total reads for each I7 index'
            )
fig.show()

fig2 = px.scatter(demultiplex_data, 
           x='n_reads', 
           y='% Perfect Index Reads', 
           color='i7', 
           hover_data=['well_letter', 'well_number'],
           title='The % of perfect index reads against total reads for each i7 index',
           template='simple_white')

fig2.show()

In [ ]:
def plot_96well_plate(metadata, color_var='mapped_reads', title='Plate A - Number of mapped reads'):
    fig = px.scatter(metadata[::-1], 
                     y='well_letter', 
                     x='well_number',
                     color=color_var, 
                     hover_data=metadata.columns, 
                     template='plotly_white')
    fig.update_traces(marker_size=40)
    fig.update_layout(xaxis = dict(
                                side='top',
                                tickmode = 'linear',
                                tick0 = 0,
                                dtick = 1), 
                      title=title)
    return fig


if plate_info:
    from IPython.display import display, Markdown
    display(Markdown('#### Visualising edge effects'))
    tot_per_well = demultiplex_data.groupby(['well_letter', 'well_number']).agg({'n_reads':'mean'}).reset_index()

    plot_96well_plate(tot_per_well,
                    color_var='n_reads',
                    title="Visualising edge effects - total reads across i7s")